In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Activation, Concatenate, Dense, Reshape, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import CSVLogger

from utils.preprocess import *
from utils.metrics import *

# Dataset


In [ ]:
X_train = np.load('X_train_Seg_op_2.npy')
Y_train = np.load('Y_train_Seg_op_2.npy')
X_test = np.load('X_test_Seg_op_2.npy')
Y_test = np.load('Y_test_Seg_op_2.npy')

## Preprocess

In [ ]:
X_train=min_max_preprocessing(X_train)
X_test=min_max_preprocessing(X_test)

In [ ]:
X_train,mean,std=samplewise_preprocessing(X_train)
X_test=featurewise_preprocessing(X_test,mean,std)

## Models

In [ ]:
#Double Unet
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = Multiply()([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []

    model = VGG19(include_top=False, weights='imagenet', input_tensor=inputs)
    names = ["block1_conv2", "block2_conv2", "block3_conv4", "block4_conv4"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("block5_conv4").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [256, 128, 64, 32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

# def encoder2(inputs):
#     skip_connections = []
#
#     output = DenseNet121(include_top=False, weights='imagenet')(inputs)
#     model = tf.keras.models.Model(inputs, output)
#
#     names = ["input_2", "conv1/relu", "pool2_conv", "pool3_conv"]
#     for name in names:
#         skip_connections.append(model.get_layer(name).output)
#     output = model.get_layer("pool4_conv").output
#
#     return output, skip_connections

def encoder2(inputs):
    num_filters = [32, 64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    #outputs = Concatenate()([outputs1, outputs2])

    model = Model(inputs, outputs2)
    return model




In [ ]:
def Unet_1():
  inputs = Input(shape=(224, 224, 1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.Dropout(0.1)(c1)
  c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.Dropout(0.1)(c2)
  c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.Dropout(0.2)(c3)
  c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.Dropout(0.2)(c4)
  c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
  
  c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.Dropout(0.3)(c5)
  c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

  #Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, c4])
  c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = tf.keras.layers.Dropout(0.2)(c6)
  c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
  
  u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c3])
  c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = tf.keras.layers.Dropout(0.2)(c7)
  c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
  
  u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c2])
  c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = tf.keras.layers.Dropout(0.1)(c8)
  c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
  
  u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
  c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
  c9 = tf.keras.layers.Dropout(0.1)(c9)
  c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Unet_2():
  inputs = Input(shape=(224, 224, 1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(112, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.Dropout(0.1)(c1)
  c1 = tf.keras.layers.Conv2D(112, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(224, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.Dropout(0.1)(c2)
  c2 = tf.keras.layers.Conv2D(224, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(448, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.Dropout(0.2)(c3)
  c3 = tf.keras.layers.Conv2D(448, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(448, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.Dropout(0.2)(c4)
  c4 = tf.keras.layers.Conv2D(448, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  

  #Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(224, (2, 2), strides=(2, 2), padding='same')(c4)
  u6 = tf.keras.layers.concatenate([u6, c3])
  c6 = tf.keras.layers.Conv2D(224, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = tf.keras.layers.Dropout(0.2)(c6)
  c6 = tf.keras.layers.Conv2D(224, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
  
  u7 = tf.keras.layers.Conv2DTranspose(112, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c2])
  c7 = tf.keras.layers.Conv2D(112, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = tf.keras.layers.Dropout(0.2)(c7)
  c7 = tf.keras.layers.Conv2D(112, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
  
  u8 = tf.keras.layers.Conv2DTranspose(112, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c1])
  c8 = tf.keras.layers.Conv2D(112, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = tf.keras.layers.Dropout(0.1)(c8)
  c8 = tf.keras.layers.Conv2D(112, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c8)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Unet_3():
  inputs = Input(shape=(224, 224, 1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.Dropout(0.1)(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.Dropout(0.1)(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.Dropout(0.2)(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.Dropout(0.2)(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
  
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.Dropout(0.3)(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

  #Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
  u6 = tf.keras.layers.concatenate([u6, c4])
  c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = tf.keras.layers.Dropout(0.2)(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
  
  u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
  u7 = tf.keras.layers.concatenate([u7, c3])
  c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = tf.keras.layers.Dropout(0.2)(c7)
  c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
  
  u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
  u8 = tf.keras.layers.concatenate([u8, c2])
  c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = tf.keras.layers.Dropout(0.1)(c8)
  c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
  
  u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
  u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
  c9 = tf.keras.layers.Dropout(0.1)(c9)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Vggunet():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Dropout(0.5)(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Dropout(0.5)(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  

  c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Dropout(0.5)(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Dropout(0.5)(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Dropout(0.5)(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Dropout(0.5)(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
  
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Dropout(0.5)(c5)
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Dropout(0.5)(c5)
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p5)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Dropout(0.5)(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)

  #Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c6)
  u6 = tf.keras.layers.concatenate([u6, c5])
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u6)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Dropout(0.5)(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)

  u7 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c7)
  u7 = tf.keras.layers.concatenate([u7, c4])
  c8 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u7)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Dropout(0.5)(c8)
  c8 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u8 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c8)
  u8 = tf.keras.layers.concatenate([u8, c3])
  c9 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u8)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  c9 = tf.keras.layers.Dropout(0.5)(c9)
  c9 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  
  u9 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c9)
  u9 = tf.keras.layers.concatenate([u9, c2])
  c10 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u9)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  c10 = tf.keras.layers.Dropout(0.5)(c10)
  c10 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c10)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  
  u10 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c10)
  u10 = tf.keras.layers.concatenate([u10, c1])
  c11 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u10)
  c11 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c11)
  c11 = tf.keras.layers.ReLU()(c11)
  c11 = tf.keras.layers.Dropout(0.5)(c11)
  c11 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c11)
  c11 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c11)
  c11 = tf.keras.layers.ReLU()(c11)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c11)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Segnet1():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  

  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  #Decoder path 
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  u1 = tf.keras.layers.UpSampling2D(size = (2, 2))(c5)

  
  c6 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  u2 = tf.keras.layers.UpSampling2D(size = (2, 2))(c6)
  
  c7 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  u3 = tf.keras.layers.UpSampling2D(size = (2, 2))(c7)
  
  c8 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  u4 = tf.keras.layers.UpSampling2D(size = (2, 2))(c8)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(u4)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Segnet1_V2():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  

  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  #Decoder path 
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  u1 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)

  
  c6 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  u2 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
  
  c7 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  u3 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
  
  c8 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  u4 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(u4)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Segnet2():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

  #Decoder path 
  u1 = tf.keras.layers.UpSampling2D(size = (2, 2))(p5)
  c6 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  
  u2 = tf.keras.layers.UpSampling2D(size = (2, 2))(c6)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  
  u3 = tf.keras.layers.UpSampling2D(size = (2, 2))(c7)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u4 = tf.keras.layers.UpSampling2D(size = (2, 2))(c8)
  c9 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u4)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  c9 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)

  u5 = tf.keras.layers.UpSampling2D(size = (2, 2))(c9)
  c10 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u5)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  c10 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c10)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Segnet2_V2():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

  #Decoder path 
  u1 = tf.keras.layers.Conv2DTranspose(1024, (2, 2), strides=(2, 2), padding='same')(p5)
  c6 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  
  u2 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c6)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  
  u3 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c7)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u4 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c8)
  c9 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u4)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  c9 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)

  u5 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c9)
  c10 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u5)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  c10 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c10)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Segnet3():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

  #Decoder path 
  u1 = tf.keras.layers.UpSampling2D(size = (2, 2))(p5)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  
  u2 = tf.keras.layers.UpSampling2D(size = (2, 2))(c6)
  c7 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  
  u3 = tf.keras.layers.UpSampling2D(size = (2, 2))(c7)
  c8 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u4 = tf.keras.layers.UpSampling2D(size = (2, 2))(c8)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u4)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  c9 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)

  u5 = tf.keras.layers.UpSampling2D(size = (2, 2))(c9)
  c10 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(u5)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  c10 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same')(c10)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()

  return model

In [ ]:
def Segnet4():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5 = tf.keras.layers.Conv2D(2048, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(2048, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(2048, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

  #Decoder path 
  u1 = tf.keras.layers.UpSampling2D(size = (2, 2))(p5)
  c6 = tf.keras.layers.Conv2D(2048, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(2048, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(2048, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  
  u2 = tf.keras.layers.UpSampling2D(size = (2, 2))(c6)
  c7 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  
  u3 = tf.keras.layers.UpSampling2D(size = (2, 2))(c7)
  c8 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u4 = tf.keras.layers.UpSampling2D(size = (2, 2))(c8)
  c9 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u4)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  c9 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)

  u5 = tf.keras.layers.UpSampling2D(size = (2, 2))(c9)
  c10 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u5)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  c10 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c10)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def Pspnet():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.LeakyReLU(alpha = 0.2)(c1)
  c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(2, 2))(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.LeakyReLU(alpha = 0.2)(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  cs1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  cs1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(cs1)
  c1 = tf.keras.layers.Add()([c1,cs1])
  c1 = tf.keras.layers.ReLU()(c1)
  
  c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(c1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.LeakyReLU(alpha = 0.2)(c2)
  c2 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(2, 2))(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.LeakyReLU(alpha = 0.2)(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  cs2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  cs2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(cs2)
  c2 = tf.keras.layers.Add()([c2,cs2])
  c2 = tf.keras.layers.ReLU()(c2)

  c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(c2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.LeakyReLU(alpha = 0.2)(c3)
  c3 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(2, 2))(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.LeakyReLU(alpha = 0.2)(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  cs3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  cs3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(cs3)
  c3 = tf.keras.layers.Add()([c3,cs3])
  c3 = tf.keras.layers.ReLU()(c3)

  #Piramid Feature Map
  r = tf.keras.layers.GlobalAveragePooling2D()(c3) 
  r = tf.keras.layers.Reshape((1,1,256))(r)
  r = tf.keras.layers.Conv2D(64, (1, 1))(r)
  r = tf.keras.layers.UpSampling2D(224,interpolation='bilinear')(r)
  
  y = tf.keras.layers.AveragePooling2D((2,2))(c3)
  y = tf.keras.layers.Conv2D(64,(1,1))(y)
  y = tf.keras.layers.UpSampling2D(2,interpolation='bilinear')(y)
  
  b = tf.keras.layers.AveragePooling2D((4,4))(c3)
  b = tf.keras.layers.Conv2D(64,(1,1))(b)
  b = tf.keras.layers.UpSampling2D(4,interpolation='bilinear')(b)
    
  g = tf.keras.layers.AveragePooling2D((8,8))(c3)
  g = tf.keras.layers.Conv2D(64,(1,1))(g)
  g = tf.keras.layers.UpSampling2D(8,interpolation='bilinear')(g)
  
  P = tf.keras.layers.concatenate([c3,r,y,b,g])
  #P = tf.keras.layers.Add()([c3,P])

  P = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(P)
  P = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(P)
  P = tf.keras.layers.LeakyReLU(alpha = 0.2)(P)
  P = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same',dilation_rate=(1, 1))(P)
  P = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(P)
  P = tf.keras.layers.LeakyReLU(alpha = 0.2)(P)
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(P)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  opt = tf.keras.optimizers.SGD(learning_rate=0.1)
  model.compile(optimizer= opt, loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

FC-DenseNet (Tiramisu-net)

In [ ]:
def BN_ReLU_Conv(inputs, n_filters, filter_size=3, dropout_p=0.2):
    '''Apply successivly BatchNormalization, ReLu nonlinearity, Convolution and Dropout (if dropout_p > 0)''' 

    l = tf.keras.layers.BatchNormalization()(inputs)
    l = tf.keras.layers.Activation('relu')(l)
    l = tf.keras.layers.Conv2D(n_filters, filter_size, padding='same', kernel_initializer='he_uniform')(l)
    if dropout_p != 0.0:
        l = tf.keras.layers.Dropout(dropout_p)(l)
    return l

def TransitionDown(inputs, n_filters, dropout_p=0.2):
    """ Apply first a BN_ReLu_conv layer with filter size = 1, and a max pooling with a factor 2  """
    l = BN_ReLU_Conv(inputs, n_filters, filter_size=1, dropout_p=dropout_p)
    l = tf.keras.layers.MaxPooling2D((2,2))(l)
    return l

def TransitionUp(skip_connection, block_to_upsample, n_filters_keep):
    '''Performs upsampling on block_to_upsample by a factor 2 and concatenates it with the skip_connection'''
    #Upsample and concatenate with skip connection
    l = tf.keras.layers.Conv2DTranspose(n_filters_keep, kernel_size=3, strides=2, padding='same', kernel_initializer='he_uniform')(block_to_upsample)
    l = tf.keras.layers.concatenate([l, skip_connection], axis=-1)
    return l

def OutputLayer(inputs,n_classes):
    """
    Performs 1x1 convolution followed by softmax nonlinearity
    The output will have the shape (batch_size  * n_rows * n_cols, n_classes)
    """
    l = tf.keras.layers.Conv2D(n_classes, kernel_size=1, padding='same', kernel_initializer='he_uniform')(inputs)
#    l = Reshape((-1, n_classes))(l)
    #l = Activation('sigmoid')(l)#or softmax for multi-class
    return l

def Tiramisu(
        input_shape=(224, 224,1),
        n_classes = 1,
        n_filters_first_conv = 48,
        n_pool = 5,
        growth_rate = 16 ,
        n_layers_per_block = [4,5,7,10,12,15,12,10,7,5,4],
        dropout_p = 0.2
        ):
    if type(n_layers_per_block) == list:
            print(len(n_layers_per_block))
    elif type(n_layers_per_block) == int:
            n_layers_per_block = [n_layers_per_block] * (2 * n_pool + 1)
    else:
        raise ValueError
        
#####################
# First Convolution #
#####################        
    inputs = Input(shape=input_shape)
    stack = tf.keras.layers.Conv2D(filters=n_filters_first_conv, kernel_size=3, padding='same', kernel_initializer='he_uniform')(inputs)
    n_filters = n_filters_first_conv

#####################
# Downsampling path #
#####################     
    skip_connection_list = []
    
    for i in range(n_pool):
        for j in range(n_layers_per_block[i]):
            l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
            stack = tf.keras.layers.concatenate([stack, l])
            n_filters += growth_rate
        
        skip_connection_list.append(stack)        
        stack = TransitionDown(stack, n_filters, dropout_p)
    skip_connection_list = skip_connection_list[::-1]

    
#####################
#    Bottleneck     #
#####################     
    block_to_upsample=[]
    
    for j in range(n_layers_per_block[n_pool]):
        l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
        block_to_upsample.append(l)
        stack = tf.keras.layers.concatenate([stack,l])
    block_to_upsample = tf.keras.layers.concatenate(block_to_upsample)

   
#####################
#  Upsampling path  #
#####################
    for i in range(n_pool):
        n_filters_keep = growth_rate * n_layers_per_block[n_pool + i ]
        stack = TransitionUp(skip_connection_list[i], block_to_upsample, n_filters_keep)
        
        block_to_upsample = []
        for j in range(n_layers_per_block[ n_pool + i + 1 ]):
            l = BN_ReLU_Conv(stack, growth_rate, dropout_p=dropout_p)
            block_to_upsample.append(l)
            stack = tf.keras.layers.concatenate([stack, l])
        block_to_upsample = tf.keras.layers.concatenate(block_to_upsample)

#####################
#  Final         #
#####################
    output = OutputLayer(stack,n_classes)            
    model=Model(inputs = inputs, outputs = output) 
    model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])   
    model.summary()
    
    return model

In [ ]:
def scaling(xx, ss=1):
            return xx * ss

def FCN():
  inputs = Input(shape=(224, 224,1), name='input')
  
  #Encoding
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c1 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c1 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

  c3 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  #c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

  c3 = tf.keras.layers.Conv2D(filters=4096, kernel_size=(7, 7), activation='relu', padding='same', dilation_rate=(2, 2), kernel_initializer='he_normal')(c3)
  c3 = tf.keras.layers.Dropout(0.5)(c3)
  c3 = tf.keras.layers.Conv2D(filters=4096, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='he_normal')(c3)
  c3 = tf.keras.layers.Dropout(0.5)(c3)

  p1 = tf.keras.layers.Conv2D(1, kernel_size=(1, 1), activation='linear', kernel_initializer='he_normal',name='p1')(c1)
  p1 = tf.keras.layers.Lambda(scaling, arguments={'ss': 1e-4})(p1)

  p2 = tf.keras.layers.Conv2D(1, kernel_size=(1, 1), activation='linear', kernel_initializer='he_normal',name='p2')(c2)
  p2 = tf.keras.layers.Lambda(scaling, arguments={'ss': 1e-2})(p2)

  p3 = tf.keras.layers.Conv2D(1, kernel_size=(1, 1), activation='linear', kernel_initializer='he_normal',name='p3')(c3)
  p3 = tf.keras.layers.Lambda(scaling, arguments={'ss': 1})(p3)

  c4 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='valid', kernel_initializer='he_normal',use_bias=False)(p3)
  c4 = tf.keras.layers.Add()([c4,p2])
  c4 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='valid', kernel_initializer='he_normal',use_bias=False)(c4)
  c4 = tf.keras.layers.Add()([c4,p1])
  c4 = tf.keras.layers.Conv2DTranspose(256, kernel_size=(2, 2), strides =(2, 2), padding='valid', kernel_initializer='he_normal',use_bias=False)(c4)
  c4 = tf.keras.layers.Conv2DTranspose(512, kernel_size=(2, 2), strides =(2, 2), padding='valid', kernel_initializer='he_normal',use_bias=False)(c4)
  c4 = tf.keras.layers.Conv2DTranspose(512, kernel_size=(2, 2), strides =(2, 2), padding='valid', kernel_initializer='he_normal',use_bias=False)(c4)

  outputs = outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

In [ ]:
def VGGUnet_pretrained():
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

    # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
    # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
    # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

    # getting weights of block1 conv1.
    block1_conv1 = model.get_layer('block1_conv1').get_weights()
    weights, biases = block1_conv1

    # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
    # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
    weights = np.transpose(weights, (3, 2, 0, 1))


    kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

    # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
    grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

      # iterate out_channels number of times
    for i in range(kernel_out_channels):

        # get kernel for every out_channel
        get_kernel = weights[i, :, :, :]

        temp_kernel = np.zeros((3, 3))

        # :get_kernel shape = [3, 3, 3]
        # axis, dims = (0, in_channel), (1, row), (2, col)

        # calculate weighted average across channel axis
        in_channels, in_rows, in_columns = get_kernel.shape

        for in_row in range(in_rows):
            for in_col in range(in_columns):
                feature_red = get_kernel[0, in_row, in_col]
                feature_green = get_kernel[1, in_row, in_col]
                feature_blue = get_kernel[2, in_row, in_col]

            # weighted average for RGB filter
            total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

            temp_kernel[in_row, in_col] = total


        # :temp_kernel is a 3x3 matrix [rows x columns]
        # add an axis at the end to specify in_channel as 1

        # Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
        temp_kernel = np.expand_dims(temp_kernel, axis=0)

        # Now, :temp_kernel shape is [1, 3, 3]

        # Concat :temp_kernel to :grayscale_weights along axis=0
        grayscale_weights[i, :, :, :] = temp_kernel

      # Dimension of :grayscale_weights is [64, 1, 3, 3]
      # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

      # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
      # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

      # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
    grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

      # combine :grayscale_weights and :biases
    new_block1_conv1 = [grayscale_weights, biases]


      # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

      # get weights of all the layers starting from 'block1_conv2'
    vgg16_weights = {}
    for layer in model.layers[2:]:
        if "conv" in layer.name:
            vgg16_weights["224_" + layer.name] = model.get_layer(layer.name).get_weights()

    del model


      # Custom build VGG16
    input = Input(shape=(224, 224, 1), name='224_input')
      # Block 1
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 1), data_format="channels_last", name='224_block1_conv1')(input)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block1_conv2')(c1)
    p1 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block1_pool')(c1)

      # Block 2
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv1')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv2')(c2)
    p2 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block2_pool')(c2)

      # Block 3
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv1')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv2')(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv3')(c3)
    p3 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block3_pool')(c3)

      # Block 4
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv1')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv2')(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv3')(c4)
    p4 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block4_pool')(c4)

      # Block 5
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv1')(p4)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv2')(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv3')(c5)
    p5 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block5_pool')(c5)

    base_model = Model(inputs=input, outputs=p5)

    base_model.get_layer('224_block1_conv1').set_weights(new_block1_conv1)
    for layer in base_model.layers[2:]:
        if 'conv' in layer.name:
            base_model.get_layer(layer.name).set_weights(vgg16_weights[layer.name])

    x = base_model.output

    for layer in base_model.layers:
        layer.trainable = True

    cb = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_blockb_conv1')(x)
    cb = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_blockb_conv2')(cb)

    u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', name='224_block6_upsam')(cb)
    u6 = tf.keras.layers.concatenate([u6, c5], name='224_block6_concat')
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block6_conv1')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block6_conv2')(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block6_conv3')(c6)
  
    u7 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', name='224_block7_upsam')(c6)
    u7 = tf.keras.layers.concatenate([u7, c4], name='224_block7_concat')
    c7 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block7_conv1')(u7)
    c7 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block7_conv2')(c7)
    c7 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block7_conv3')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name='224_block8_upsam')(c7)
    u8 = tf.keras.layers.concatenate([u8, c3], name='224_block8_concat')
    c8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block8_conv1')(u8)
    c8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block8_conv2')(c8)
    c8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block8_conv3')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='224_block9_upsam')(c8)
    u9 = tf.keras.layers.concatenate([u9, c2], name='224_block9_concat')
    c9 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block9_conv1')(u9)
    c9 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block9_conv2')(c9)

    u10 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='224_block10_upsam')(c9)
    u10 = tf.keras.layers.concatenate([u10, c1], name='224_block10_concat')
    c10 = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block10_conv1')(u10)
    c10 = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block10_conv2')(c10)
      
    predictions = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
    #Compilador
    model = tf.keras.Model(inputs=[base_model.input], outputs=[predictions])
    model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
    model.summary()
    return model

In [ ]:
def VGGSegnet():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Encoder path
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
  
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
  
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  c5 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
  c5 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c5)
  c5 = tf.keras.layers.ReLU()(c5)
  p5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c5)

  #Decoder path 
  u1 = tf.keras.layers.UpSampling2D(size = (2, 2))(p5)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  c6 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
  c6 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c6)
  c6 = tf.keras.layers.ReLU()(c6)
  
  u2 = tf.keras.layers.UpSampling2D(size = (2, 2))(c6)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)
  
  u3 = tf.keras.layers.UpSampling2D(size = (2, 2))(c7)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  c8 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u4 = tf.keras.layers.UpSampling2D(size = (2, 2))(c8)
  c9 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u4)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  c9 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)

  u5 = tf.keras.layers.UpSampling2D(size = (2, 2))(c9)
  c10 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u5)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  c10 = tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c10)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()

  return model

In [ ]:
def VGGSegnet_pretrained():
    model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

    # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
    # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
    # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

    # getting weights of block1 conv1.
    block1_conv1 = model.get_layer('block1_conv1').get_weights()
    weights, biases = block1_conv1

    # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
    # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
    weights = np.transpose(weights, (3, 2, 0, 1))


    kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

    # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
    grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

      # iterate out_channels number of times
    for i in range(kernel_out_channels):

        # get kernel for every out_channel
        get_kernel = weights[i, :, :, :]

        temp_kernel = np.zeros((3, 3))

        # :get_kernel shape = [3, 3, 3]
        # axis, dims = (0, in_channel), (1, row), (2, col)

        # calculate weighted average across channel axis
        in_channels, in_rows, in_columns = get_kernel.shape

        for in_row in range(in_rows):
            for in_col in range(in_columns):
                feature_red = get_kernel[0, in_row, in_col]
                feature_green = get_kernel[1, in_row, in_col]
                feature_blue = get_kernel[2, in_row, in_col]

            # weighted average for RGB filter
            total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

            temp_kernel[in_row, in_col] = total


        # :temp_kernel is a 3x3 matrix [rows x columns]
        # add an axis at the end to specify in_channel as 1

        # Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
        temp_kernel = np.expand_dims(temp_kernel, axis=0)

        # Now, :temp_kernel shape is [1, 3, 3]

        # Concat :temp_kernel to :grayscale_weights along axis=0
        grayscale_weights[i, :, :, :] = temp_kernel

      # Dimension of :grayscale_weights is [64, 1, 3, 3]
      # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

      # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
      # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

      # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
    grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

      # combine :grayscale_weights and :biases
    new_block1_conv1 = [grayscale_weights, biases]


      # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

      # get weights of all the layers starting from 'block1_conv2'
    vgg16_weights = {}
    for layer in model.layers[2:]:
        if "conv" in layer.name:
            vgg16_weights["224_" + layer.name] = model.get_layer(layer.name).get_weights()

    del model


      # Custom build VGG16
    input = Input(shape=(224, 224, 1), name='224_input')
      # Block 1
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 1), data_format="channels_last", name='224_block1_conv1')(input)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block1_conv2')(c1)
    p1 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block1_pool')(c1)

      # Block 2
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv1')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block2_conv2')(c2)
    p2 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block2_pool')(c2)

      # Block 3
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv1')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv2')(c3)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block3_conv3')(c3)
    p3 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block3_pool')(c3)

      # Block 4
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv1')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv2')(c4)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block4_conv3')(c4)
    p4 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block4_pool')(c4)

      # Block 5
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv1')(p4)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv2')(c5)
    c5 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block5_conv3')(c5)
    p5 = MaxPooling2D((2, 2), strides=(2, 2), name='224_block5_pool')(c5)

    base_model = Model(inputs=input, outputs=p5)

    base_model.get_layer('224_block1_conv1').set_weights(new_block1_conv1)
    for layer in base_model.layers[2:]:
        if 'conv' in layer.name:
            base_model.get_layer(layer.name).set_weights(vgg16_weights[layer.name])

    x = base_model.output

    for layer in base_model.layers:
        layer.trainable = True

    cb = Conv2D(1024, (3, 3), activation='relu', padding='same', name='224_blockb_conv1')(x)
    cb = Conv2D(1024, (3, 3), activation='relu', padding='same', name='224_blockb_conv2')(cb)

    u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', name='224_block6_upsam')(cb)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block6_conv1')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block6_conv2')(c6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block6_conv3')(c6)
  
    u7 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same', name='224_block7_upsam')(c6)
    c7 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block7_conv1')(u7)
    c7 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block7_conv2')(c7)
    c7 = Conv2D(512, (3, 3), activation='relu', padding='same', name='224_block7_conv3')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same', name='224_block8_upsam')(c7)
    c8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block8_conv1')(u8)
    c8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block8_conv2')(c8)
    c8 = Conv2D(256, (3, 3), activation='relu', padding='same', name='224_block8_conv3')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='224_block9_upsam')(c8)
    c9 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block9_conv1')(u9)
    c9 = Conv2D(128, (3, 3), activation='relu', padding='same', name='224_block9_conv2')(c9)

    u10 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same', name='224_block10_upsam')(c9)
    c10 = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block10_conv1')(u10)
    c10 = Conv2D(64, (3, 3), activation='relu', padding='same', name='224_block10_conv2')(c10)
      
    predictions = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
    #Compilador
    model = tf.keras.Model(inputs=[base_model.input], outputs=[predictions])
    model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
    model.summary()
    return model

In [ ]:
def MobielenetUnet():
  inputs = Input(shape=(224, 224,1), name='input')
  #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

  #Contraction path
  c1 = tf.keras.layers.Conv2D(32, (3, 3), kernel_initializer='he_normal', padding='same',strides=(2,2))(inputs)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Dropout(0.5)(c1)

  c1 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=32,strides=(1,1),use_bias=False)(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)
  c1 = tf.keras.layers.Conv2D(64, (1, 1), kernel_initializer='he_normal', padding='same')(c1)
  c1 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c1)
  c1 = tf.keras.layers.ReLU()(c1)

  c2 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=64,strides=(2,2),use_bias=False)(c1)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(128, (1, 1), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  
  c2 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=128,strides=(1,1),use_bias=False)(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)
  c2 = tf.keras.layers.Conv2D(128, (1, 1), kernel_initializer='he_normal', padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c2)
  c2 = tf.keras.layers.ReLU()(c2)

  c3 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=128,strides=(2,2),use_bias=False)(c2)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (1, 1), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)

  c3 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=256,strides=(1,1),use_bias=False)(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  c3 = tf.keras.layers.Conv2D(256, (1, 1), kernel_initializer='he_normal', padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c3)
  c3 = tf.keras.layers.ReLU()(c3)
  
  
  c4 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=256,strides=(2,2),use_bias=False)(c3)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (1, 1), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)

  c4 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=512,strides=(1,1),use_bias=False)(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (1, 1), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)

  c4 = tf.keras.layers.DepthwiseConv2D((3,3), padding='same',depth_multiplier=512,strides=(1,1),use_bias=False)(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)
  c4 = tf.keras.layers.Conv2D(512, (1, 1), kernel_initializer='he_normal', padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c4)
  c4 = tf.keras.layers.ReLU()(c4)

#Expansive path 
  u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c4)
  u6 = tf.keras.layers.concatenate([u6, c3])
  c7 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u6)
  c7 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c7)
  c7 = tf.keras.layers.ReLU()(c7)

  u7 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c7)
  u7 = tf.keras.layers.concatenate([u7, c2])
  c8 = tf.keras.layers.Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u7)
  c8 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c8)
  c8 = tf.keras.layers.ReLU()(c8)
  
  u8 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c8)
  u8 = tf.keras.layers.concatenate([u8, c1])
  c9 = tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u8)
  c9 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c9)
  c9 = tf.keras.layers.ReLU()(c9)
  
  u9 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c9)
  c10 = tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u9)
  c10 = tf.keras.layers.BatchNormalization(momentum=0.2, epsilon=0.001, center=True, scale=False, trainable=True, fused=None, renorm=False, renorm_clipping=None, renorm_momentum=0.4, adjustment=None)(c10)
  c10 = tf.keras.layers.ReLU()(c10)  
  
  outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c10)
  
  model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam', loss=dice_coef_loss, metrics=['accuracy',dice_coef,iou,])
  model.summary()
  return model

# Train

## Metrics

In [ ]:
model = MobielenetUnet()

In [ ]:
model_name='Opa_Seg_MobilenetUnet_B4_Dataset_2'
log_dir="logs/"
filepath = log_dir+"Saved_models/"+model_name+".h5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
csv_logger = CSVLogger('logs/csv/'+model_name+'.csv', append=False, separator=';')
history=model.fit(X_train, Y_train, epochs=200, batch_size=2, validation_data=(X_test, Y_test),callbacks=[csv_logger,checkpoint])

In [ ]:
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper left')

In [ ]:
plt.figure()
plt.plot(history.history['dice_coef'])
plt.plot(history.history['val_dice_coef'])
plt.title('model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')

In [ ]:
g=pd.read_csv('logs/csv/'+model_name+'.csv', sep=';')
for i in range(len(g)):
  if g['val_dice_coef'][i]==np.max(g['val_dice_coef']):
    print('index =',i)
    print('dice=',g['dice_coef'][i])
    print('Iou=',g['iou'][i])
    print('val dice=',g['val_dice_coef'][i])
    print('val_Iou=',g['val_iou'][i])

# Metrics

In [ ]:
model.load_weights('logs/Saved_models/'+model_name+'.h5')

In [ ]:
preds_test = model.predict(X_test)
pred_img=[]
for i in preds_test:
  pred_img.append(np.resize(i,(224,224)))

In [ ]:
plt.subplot(1,3,1)
plt.imshow(X_test[2],cmap='gray')
plt.subplot(1,3,2)
plt.imshow(Y_test[2],cmap='gray')
plt.subplot(1,3,3)
plt.imshow(pred_img[2],cmap='gray')

In [ ]:
plt.subplot(1,3,1)
plt.imshow(X_test[1])
plt.subplot(1,3,2)
plt.imshow(Y_test[1])
plt.subplot(1,3,3)
plt.imshow(pred_img[1])